In [6]:
#Requires to have geopandas installed
#Installation https://geopandas.org/getting_started/install.html

import matplotlib.pyplot as plt 
import geopandas as gpd
import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
from pandas import json_normalize
import requests # convert json into dataframe
#For OSM 
import pyrosm
from pyrosm import OSM, get_data

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

#  Adding MENA to Africa

In [3]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [4]:
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

In [5]:
africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [6]:
### Example plot https://geopandas.org/gallery/plotting_with_geoplot.html?highlight=africa

# #gdp_per_cap = world.gdp_md_est / world.pop_est
# ax = geoplot.cartogram(
#     africa, scale='pop_est', limits=(0.2, 1),
#     edgecolor='None', figsize=(7, 8)
# )
# geoplot.polyplot(africa, edgecolor='gray', ax=ax)

In [7]:
#ax = africa.plot(color="white", edgecolors="red",figsize=(10, 10))

# 3) Substations - Onsset - OSM

Install PYROSM to handle OSM data: https://pyrosm.readthedocs.io/en/latest/installation.html

data source http://download.geofabrik.de/  

data description: https://gep-onsset.readthedocs.io/en/latest/GIS%20data%20collection.html

### OSM content check

In [8]:
from pyrosm import OSM, get_data

# Initialize the OSM reader with test data
fp = get_data("Helsinki")
osm = OSM(fp)

# The instance has a configuration attribute containing:
print([item for item in osm.conf.__dict__.keys() if not item.startswith("_")])


Downloaded Protobuf data 'Helsinki.osm.pbf' (29.27 MB) to:
'/tmp/pyrosm/Helsinki.osm.pbf'
['network_filters', 'tags', 'oneway_values']


In [9]:
# Show all available tag attributes
#osm.conf.tags.available


In [10]:
# Show all tags that are converted into columns from building features
osm.conf.tags.power

['cable',
 'catenary_mast',
 'compensator',
 'converter',
 'generator',
 'heliostat',
 'insulator',
 'line',
 'busbar',
 'bay',
 'minor_line',
 'plant',
 'pole',
 'portal',
 'power',
 'substation',
 'switch',
 'switchgear',
 'terminal',
 'tower',
 'transformer']

## Create OSM substation dataframe

Pyrosm has an existing issue handling geofabric data. Thats why "get_data("africa")" is not working. 

A way around this problem is to download the data from http://download.geofabrik.de/africa-latest.osm.bz2 locally & set the path right

://github.com/HTenkanen/pyrosm/issues/109

In [11]:
pwd

'/home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/pypsa-africa/data_exploration'

In [12]:
cd

/home/max


In [13]:
cd "OneDrive/PHD-Flexibility/07_pypsa-africa/Data"

/home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/Data


In [14]:
africa_data = "africa-latest.osm.pbf"

In [15]:
#Possible input to extract data from PYROSM. I.e. get_data("angola") gets the "angola-latest.osm.pbf" filled with all OSM data
regions =     ["algeria",
               "angola",
               "benin",
               "botswana",
               "burkina_faso",
               "burundi",
               "cameroon",
               "canary_islands",
               "cape_verde",
               "central_african_republic",
               "chad",
               "comores",
               "congo_brazzaville",
               "congo_democratic_republic",
               "djibouti",
               "egypt",
               "equatorial_guinea",
               "eritrea",
               "ethiopia",
               "gabon",
               "ghana",
               "guinea_bissau",
               "guinea",
               "ivory_coast",
               "kenya",
               "lesotho",
               "liberia",
               "libya",
               "madagascar",
               "malawi",
               "mali",
               "mauritania",
               "mauritius",
               "morocco",
               "mozambique",
               "namibia",
               "niger",
               "nigeria",
               "rwanda",
               "saint_helena_ascension_and_tristan_da_cunha",
               "sao_tome_and_principe",
               "senegal_and_gambia",
               "seychelles",
               "sierra_leone",
               "somalia",
               "south_africa_and_lesotho",
               "south_africa",
               "south_sudan",
               "sudan",
               "swaziland",
               "tanzania",
               "togo",
               "tunisia",
               "uganda",
               "zambia",
               "zimbabwe"]

In [2]:
fp = get_data("lesotho")

NameError: name 'get_data' is not defined

In [3]:
fp = '/home/matin/pypsa-africa/Africa-node.osm.pbf'

In [40]:
osm = pyrosm.OSM(fp)

In [41]:
my_filter = {'power': ["substation"]}

In [1]:
lesotho = osm.get_pois(custom_filter=my_filter)

NameError: name 'osm' is not defined

In [ ]:
lesotho.head(5)

In [ ]:
#Shows geometry reference
lesotho['geometry'].crs

In [ ]:
#Converting Polygons to "centroids" = Points. 

#We used "epsg=3035" to make the long/lat numbers larger.
#This was necessary to make the centroid operations more accurate. Only required for small polygons.
#Afterward we reconvert it to the standard epsg=4326

lesotho['geometry'] = lesotho['geometry'].to_crs(epsg=3035)
lesotho['geometry'] = lesotho['geometry'].centroid
lesotho['geometry'] = lesotho['geometry'].to_crs(epsg=4326)

In [ ]:
# Plot
ax = lesotho.plot(column='power', marker='o', markersize=10, figsize=(6,6), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))

In [ ]:
#Worked! Lesotho is downn i
ax = lesotho.plot(column='power', marker='o', markersize=10, figsize=(6,6), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))
geoplot.polyplot(africa, edgecolor='gray', ax=ax)

# Overpass 

In [8]:
url = 'https://lz4.overpass-api.de/api/interpreter'  # Overpass API URL


In [4]:
# Bounding Boxes
# Nigeria (approx) (0.61,-4.21,21.75,15.79)
query = f"""
[out:json];
// gather results
(
  // query part for: “power=substation”
  ( area["ISO3166-1"="IT"];) ->.a;
  node["power"="substation"](area.a);
  way["power"="substation"](area.a);
  relation["power"="substation"](area.a);
);
// print results
out body;
"""

In [9]:
r = requests.get(url, params={'data': query})
data = r.json()['elements'] # read response as JSON and get the data
df = pd.json_normalize(data)  # create a DataFrame from the data

In [10]:
print(df.shape)

(31665, 399)


In [11]:
# Pretty Print dataframe (vscode)
display(df)

,type,id,lat,lon,tags.power,tags.operator,tags.substation,tags.building,tags.ref,tags.man_made,...,tags.name:zu,tags.official_name:cs,tags.official_name:de,tags.official_name:el,tags.official_name:eo,tags.old_name:vi,tags.population,tags.population:date,tags.source:name:oc,tags.timezone
0,node,315982776,45.704380,9.343410,substation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,353116863,45.547929,9.184324,substation,Enel,minor_distribution,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,354536567,43.360798,13.697688,substation,Enel Distrbuzione SPA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,358152934,43.413075,13.647388,substation,Enel Distribuzione SPA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,364121580,45.599718,9.878701,substation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31660,relation,11977599,NaN,NaN,substation,NaN,NaN,service,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31661,relation,12046318,NaN,NaN,substation,Acea Distribuzione,distribution,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31662,relation,12126279,NaN,NaN,substation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31663,relation,12423214,NaN,NaN,substation,Terna,transition,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
